# Imports

In [5]:
import kagglehub
import shutil
import os

# Data from TwelveData

First you have to create an account [here](https://twelvedata.com) and the generate a key [here](https://twelvedata.com/account/api-keys). Place the key in a file named .env and make sure there exists this field:

> ⚠️ **Important:** Do **not** share this key or commit the `.env` file to version control. It's already excluded in `.gitignore`.

> TD_API_KEY=<yllaertubyrtecinreadbackwards>

In [6]:
from twelvedata import TDClient
from dotenv import load_dotenv
import os

In [7]:
load_dotenv()
api_key = os.getenv("TD_API_KEY")
db_username = os.getenv("db_username")
db_password = os.getenv("db_password")

In [8]:
# Initialize client
td = TDClient(apikey=api_key)

# Fetch data
ts = td.time_series(
    symbol="AAPL",
    interval="1min",
    outputsize=100,
    timezone="America/New_York",
)

df = ts.as_pandas()
print(df.head())


InvalidApiKeyError: **apikey** parameter is incorrect or not specified. You can get your free API key instantly following this link: https://twelvedata.com/pricing. If you believe that everything is correct, you can contact us at https://twelvedata.com/contact/customer

In [ ]:
df.reset_index().to_csv("data/raw/AAPL_1min.csv", index=False)

In [ ]:
# Fetch daily time series data for SPY from 2024-01-01 to 2024-12-31
ts = td.time_series(
    symbol="SPY",
    interval="1day",
    start_date="2024-01-01",
    end_date="2024-12-31",
    timezone="America/New_York"
)

# Convert the time series data to a pandas DataFrame
df = ts.as_pandas()

# Display the first few rows of the DataFrame
print(df.head())

# Optional: Save the DataFrame to a CSV file
df.to_csv("data/raw/SPY_2024_daily.csv", index=True)

In [ ]:
# Fetch daily time series data for SPY from 2024-01-01 to 2024-12-31
ts = td.time_series(
    symbol="SPY",
    interval="1month",
    start_date="2024-01-01",
    end_date="2024-12-31",
    timezone="America/New_York"
)

# Convert the time series data to a pandas DataFrame
df = ts.as_pandas()

# Display the first few rows of the DataFrame
print(df.head())

# Optional: Save the DataFrame to a CSV file
df.to_csv("data/raw/SPY_2024_monthly.csv", index=True)

In [ ]:
# Fetch daily time series data for SPY from 2024-01-01 to 2024-12-31
ts = td.time_series(
    symbol="BA",
    interval="1month",
    start_date="2024-01-01",
    end_date="2024-12-31",
    timezone="America/New_York"
)

# Convert the time series data to a pandas DataFrame
df = ts.as_pandas()

# Display the first few rows of the DataFrame
print(df.head())

# Optional: Save the DataFrame to a CSV file
df.to_csv("data/raw/BA_2024_monthly.csv", index=True)

In [ ]:
# Fetch daily time series data for SPY from 2024-01-01 to 2024-12-31
ts = td.time_series(
    symbol="BA",
    interval="1day",
    start_date="2024-01-01",
    end_date="2024-12-31",
    timezone="America/New_York"
)

# Convert the time series data to a pandas DataFrame
df = ts.as_pandas()

# Display the first few rows of the DataFrame
print(df.head())

# Optional: Save the DataFrame to a CSV file
df.to_csv("data/raw/BA_2024_daily.csv", index=True)

# Stockmarket data

# Connecting to the DB

In [ ]:
df.columns

In [ ]:
#pip install pandas psycopg2-binary sqlalchemy

In [ ]:
import pandas as pd
import psycopg2
from psycopg2.extras import execute_values

# Database connection parameters
db_config = {
    'host': '192.168.1.68',
    'port': 5432,
    'database': 'dwtestdb',
    'user': db_username,
    'password': db_password
}

# Add the datetime column to the DataFrame
df['datetime'] = df.index

# Connect to PostgreSQL
try:
    connection = psycopg2.connect(**db_config)
    cursor = connection.cursor()
    
    # Create table
    create_table_query = """
    CREATE TABLE IF NOT EXISTS stock_data (
        id SERIAL PRIMARY KEY,
        open NUMERIC(10, 5),
        high NUMERIC(10, 5),
        low NUMERIC(10, 5),
        close NUMERIC(10, 5),
        volume BIGINT,
        datetime DATE
    );
    """
    
    cursor.execute(create_table_query)
    connection.commit()
    print("Table created successfully or already exists.")
    
    # Prepare data for insertion
    insert_query = """
    INSERT INTO stock_data (open, high, low, close, volume, datetime)
    VALUES %s
    """
    
    # Convert DataFrame to list of tuples
    data_tuples = list(df.itertuples(index=False, name=None))
    
    # Execute batch insert
    execute_values(
        cursor,
        insert_query,
        data_tuples,
        template=None,
        page_size=100
    )
    
    connection.commit()
    print(f"Successfully inserted {len(data_tuples)} rows")
    
    # Verify insertion
    cursor.execute("SELECT COUNT(*) FROM stock_data")
    count = cursor.fetchone()[0]
    print(f"Total rows in table: {count}")
    
except Exception as e:
    print(f"Error: {e}")
    connection.rollback()
    
finally:
    if connection:
        cursor.close()
        connection.close()

# Historical data

Due to the limitations of the free plan, we have to get historical data using other means, we decided Kaggle is the best way.

In [ ]:
#pip install kagglehub

## Inflation data

In [ ]:
os.getcwd()

In [ ]:


# Download latest version
dataset_path = kagglehub.dataset_download("sazidthe1/global-inflation-data")
kaggle_raw_dir = "data/raw/kaggle/inflation_world/"
print("Path to dataset files:", dataset_path)

if os.path.exists(kaggle_raw_dir):
    shutil.rmtree(kaggle_raw_dir)

# Copy the entire directory tree
shutil.copytree(dataset_path, kaggle_raw_dir)

print(f"Dataset downloaded and copied to: {kaggle_raw_dir}")

In [ ]:
import kagglehub
import shutil
import os

# Download latest version
dataset_path = kagglehub.dataset_download("varpit94/us-inflation-data-updated-till-may-2021")
kaggle_raw_dir = "/data/raw/kaggle/inflation/"
print("Path to dataset files:", dataset_path)

if os.path.exists(kaggle_raw_dir):
    shutil.rmtree(kaggle_raw_dir)

# Copy the entire directory tree
shutil.copytree(dataset_path, kaggle_raw_dir)

print(f"Dataset downloaded and copied to: {kaggle_raw_dir}")

## NYSE data

In [ ]:
# Download the dataset to a specific directory
dataset_path = kagglehub.dataset_download("svaningelgem/nyse-100-daily-stock-prices")

# If you want to copy/move it to your desired location
kaggle_raw_dir = "/data/raw/kaggle/nyse/"

# Remove target directory if it exists, then copy everything
if os.path.exists(kaggle_raw_dir):
    shutil.rmtree(kaggle_raw_dir)

# Copy the entire directory tree
shutil.copytree(dataset_path, kaggle_raw_dir)

print(f"Dataset downloaded and copied to: {kaggle_raw_dir}")

## Insert sample

In [ ]:
files = os.listdir(kaggle_raw_dir)

print("There are", len(files), "files in the directory")

# for file in files:
#     file_path = os.path.join(kaggle_raw_dir, file)
#     print(file, file_path)

sample_file = files[0]
sample_file_path = os.path.join(kaggle_raw_dir, sample_file)
pd.read_csv(sample_file_path).head().columns


In [ ]:
print(f"Reading CSV file from: {sample_file_path}")
df = pd.read_csv(sample_file_path)

# Preview the data
print("DataFrame shape:", df.shape)
print("DataFrame columns:", df.columns)
print("First 5 rows:")
print(df.head())

In [ ]:


# Connect to the PostgreSQL database
try:
    conn = psycopg2.connect(**db_config)
    cursor = conn.cursor()
    print("Successfully connected to PostgreSQL database")

    # Create the FactStock table
    create_table_query = '''
    CREATE TABLE IF NOT EXISTS FactStock (
        id SERIAL PRIMARY KEY,
        ticker VARCHAR(10) NOT NULL,
        date DATE NOT NULL,
        open NUMERIC(10, 2),
        high NUMERIC(10, 2),
        low NUMERIC(10, 2),
        close NUMERIC(10, 2)
    );
    '''
    cursor.execute(create_table_query)
    conn.commit()
    print("FactStock table created or already exists")

    # Select only the required columns
    if set(['ticker', 'date', 'open', 'high', 'low', 'close']).issubset(set(df.columns)):
        df_subset = df[['ticker', 'date', 'open', 'high', 'low', 'close']]
        
        # Convert date column to datetime if it's not already
        if not pd.api.types.is_datetime64_any_dtype(df_subset['date']):
            df_subset['date'] = pd.to_datetime(df_subset['date'])
            
        # Convert dataframe to list of tuples for faster insertion
        data_tuples = list(df_subset.itertuples(index=False, name=None))
        
        # Insert the data in batches
        insert_query = '''
        INSERT INTO FactStock (ticker, date, open, high, low, close)
        VALUES %s
        '''
        
        # Use execute_values for faster insertion
        execute_values(cursor, insert_query, data_tuples)
        conn.commit()
        
        # Count rows to verify
        cursor.execute("SELECT COUNT(*) FROM FactStock")
        row_count = cursor.fetchone()[0]
        print(f"Successfully imported {row_count} rows into FactStock table")
    else:
        print("Error: Required columns not found in the CSV file")
        print(f"Expected: ['ticker', 'date', 'open', 'high', 'low', 'close']")
        print(f"Found: {df.columns.tolist()}")

except Exception as e:
    print(f"Database Error: {e}")
    
finally:
    # Close the connection
    if 'conn' in locals() and conn is not None:
        cursor.close()
        conn.close()
        print("PostgreSQL connection closed")

## Insert all

In [ ]:
def insert_stock_data(filepath):
    df = pd.read_csv(filepath)
    try:
        conn = psycopg2.connect(**db_config)
        cursor = conn.cursor()
        print("Successfully connected to PostgreSQL database")

        # Create the FactStock table
        create_table_query = '''
        CREATE TABLE IF NOT EXISTS FactStock (
            id SERIAL PRIMARY KEY,
            ticker VARCHAR(10) NOT NULL,
            date DATE NOT NULL,
            open NUMERIC(10, 2),
            high NUMERIC(10, 2),
            low NUMERIC(10, 2),
            close NUMERIC(10, 2)
        );
        '''
        cursor.execute(create_table_query)
        conn.commit()
        print("FactStock table created or already exists")

        # Select only the required columns
        if set(['ticker', 'date', 'open', 'high', 'low', 'close']).issubset(set(df.columns)):
            df_subset = df[['ticker', 'date', 'open', 'high', 'low', 'close']]
            
            # Convert date column to datetime if it's not already
            if not pd.api.types.is_datetime64_any_dtype(df_subset['date']):
                df_subset['date'] = pd.to_datetime(df_subset['date'])
                
            # Convert dataframe to list of tuples for faster insertion
            data_tuples = list(df_subset.itertuples(index=False, name=None))
            
            # Insert the data in batches
            insert_query = '''
            INSERT INTO FactStock (ticker, date, open, high, low, close)
            VALUES %s
            '''
            
            # Use execute_values for faster insertion
            execute_values(cursor, insert_query, data_tuples)
            conn.commit()
            
            # Count rows to verify
            cursor.execute("SELECT COUNT(*) FROM FactStock")
            row_count = cursor.fetchone()[0]
            print(f"Successfully imported {row_count} rows into FactStock table")
        else:
            print("Error: Required columns not found in the CSV file")
            print(f"Expected: ['ticker', 'date', 'open', 'high', 'low', 'close']")
            print(f"Found: {df.columns.tolist()}")

    except Exception as e:
        print(f"Database Error: {e}")
        
    finally:
        # Close the connection
        if 'conn' in locals() and conn is not None:
            cursor.close()
            conn.close()
            print("PostgreSQL connection closed")

In [ ]:
for file in files:
    if file.endswith(".csv"):
        # Check if the file is a CSV file
        print(f"Processing {file}...")
        file_path = os.path.join(kaggle_raw_dir, file)
        print(file, file_path)
        insert_stock_data(file_path)

# Helper functions

## Get stock data for month

In [ ]:
def get_stock_data(symbol, interval, start_date, end_date, custom_name=None):
    """
    Fetch stock data from Twelve Data API.
    
    Parameters:
    symbol (str): Stock symbol.
    interval (str): Time interval (e.g., '1min', '1day').
    start_date (str): Start date in 'YYYY-MM-DD' format.
    end_date (str): End date in 'YYYY-MM-DD' format.
    
    Returns:
    pd.DataFrame: DataFrame containing stock data.
    """
    ts = td.time_series(
        symbol=symbol,
        interval=interval,
        start_date=start_date,
        end_date=end_date,
        timezone="America/New_York"
    )

    df = ts.as_pandas()

    file_name = f"data/raw/{symbol}_{interval}_{start_date}_{end_date}.csv"
    if custom_name:
        file_name = f"data/raw/{custom_name}.csv"

    df.to_csv(file_name, index=True)
    
    return df

In [ ]:
get_stock_data("BA", "1day", "2024-01-01", "2024-02-28")

In [ ]:
# Fetch daily time series data for SPY from 2024-01-01 to 2024-12-31
ts = td.time_series(
    symbol="BA",
    interval="1day",
    start_date="2024-01-01",
    end_date="2024-12-31",
    timezone="America/New_York"
)

# Convert the time series data to a pandas DataFrame
df = ts.as_pandas()

# Display the first few rows of the DataFrame
print(df.head())

# Optional: Save the DataFrame to a CSV file
df.to_csv("data/raw/BA_2024_daily.csv", index=True)

## Insert stock data into db

In [ ]:
def update_database(df, table_name="FactStock"):
    """Update PostgreSQL database with new data"""
    try:
        conn = psycopg2.connect(**db_config)
        cursor = conn.cursor()
        
        # Add datetime column if not present
        if 'datetime' not in df.columns:
            df['datetime'] = df.index
        
        # Prepare data for insertion
        data_tuples = list(df.itertuples(index=False, name=None))
        
        # Insert the data
        insert_query = f"""
        INSERT INTO {table_name} (ticker, date, open, high, low, close)
        VALUES %s
        ON CONFLICT (ticker, date) DO UPDATE
        SET open = EXCLUDED.open,
            high = EXCLUDED.high,
            low = EXCLUDED.low,
            close = EXCLUDED.close
        """
        
        execute_values(cursor, insert_query, data_tuples)
        conn.commit()
        print(f"Successfully updated {len(data_tuples)} rows in {table_name}")
        
    except Exception as e:
        print(f"Database Error: {e}")
        if 'conn' in locals():
            conn.rollback()
    finally:
        if 'conn' in locals():
            cursor.close()
            conn.close()